In [1]:

#------------------------------------------------------------------------
# WES 207 Demo Code
#------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

print('done')

done


In [2]:

# Load bitstream
ol = Overlay('AXI_SPI_BUS_PMOD_NAV_49.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()

print('done')
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

done
  
0.6758265495300293 seconds to program bitstream


### initializing MMIO objects for IP core debug

In [3]:
# new core test

# testing SPI output

# SPI CORE config
QUAD_SPI_BASE_ADDERSS = 0x41E00000
SPI_DRIVER_BASE_ADDRESS = 0x40000000
SPI_DRIVER_DATA_ADDRESS = 0x40004000
#SPI_DRIVER_TEST_ADDRESS = 0x40012000

#POSITION_CTRL_DATA_ADDRESS = 0x40010000
#POSITION_TEST_DATA_ADDRESS = 0x40020000
ADDRESS_RANGE = 0x1000 # 4k size
#ADDRESS_RANGE_8K = 0x2000 # 8k size
ADDRESS_RANGE_16K = 0x4000 # 16k size

# creating SPI CORE object for messages
QSPI_CORE = MMIO(QUAD_SPI_BASE_ADDERSS,ADDRESS_RANGE)#, True)
SPI_DRIVER = MMIO(SPI_DRIVER_BASE_ADDRESS,ADDRESS_RANGE)#, True)
SPI_DATA = MMIO(SPI_DRIVER_DATA_ADDRESS,ADDRESS_RANGE_16K)#,True)
#SPI_TEST = MMIO(SPI_DRIVER_TEST_ADDRESS,ADDRESS_RANGE_8K,True)

#POS_DATA = MMIO(POSITION_CTRL_DATA_ADDRESS, ADDRESS_RANGE,True)
#POS_TEST = MMIO(POSITION_TEST_DATA_ADDRESS, ADDRESS_RANGE,True)
navDataStateDemo = 0
driverStateDemo = 0
print(' ')
print('done')

 
done


## Debug Cells Below

### reading CTRL register from HLS core

In [4]:
# test reads

test = SPI_DRIVER.read(0x00)
print('\nCalling SPI_Driver.read(0x00):')
print(test)

print('done')


Calling SPI_Driver.read(0x00):
4
done


### writing ap_start and auto_restart CTRL bits in HLS core

In [ ]:
# test code

# setting ap_start/auto reset on SPI Driver
#SPI_DRIVER.write(0x00,0x81) #runs continuously, see xax_spi_driver_hw.h

# setting 
#SPI_DRIVER.write(0x00,0x01) #runs once
#SPI_DRIVER.write(0x00,0x81) #runs continuously, see xax_spi_driver_hw.h

READ = SPI_DRIVER.read(0x00)
print('\nCalling SPI_DRIVER.write(0x00,0x01)')
print('Calling SPI_Driver.read(0x00):')
print(READ)

print('done')

### test read of QUAD SPI IP core configuration registers

In [ ]:
# this is a basic configuration of the SPI core
# I would read through the docs a bit more and make sure this is correct
# it seems to work as is, 8 bit addres and 8 bit data with SS and CLK

QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,0xAA) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0

#QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0
# testing config
#QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
#QSPI_CORE.write(0x60, 0x6) # enable QSPI core in master mode, auto SS
QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
#QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0

# reading back config registers
read1 = QSPI_CORE.read(0x60)
read2 = QSPI_CORE.read(0x70)

print(read1)
print(read2)
print('done')

### Testing read from PMODNav GYRO/ACC WHO_AM_I Register 0x0F

In [74]:
# PMODNav setup, State Machine
WRITE_OFFSET = 0x68 #SPI_DTR, SPI data transmit
SPICR = 0X60
SPI_DRR = 0x6C

CTRL_REG5_XL = 0x38
CTRL_REG6_XL = 0xDB
WRITE_CFG  = 0x7F
READ_CFG = 1<<7
CTRL_REG1_G = 0x6F
CTRL_REG4 = 0x38
GYRO_WHO_AM_I = 0x0F

ADDRESS = CTRL_REG5_XL & WRITE_CFG
DATA = 0x38 # enable all 3 GYRO axis
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,MESSAGE) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0

#QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

ADDRESS = CTRL_REG6_XL & WRITE_CFG
DATA = 0xD8  # set GYRO odr to 952 Hz
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,MESSAGE) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0

#QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

ADDRESS = CTRL_REG1_G & WRITE_CFG
DATA = 0xDB
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,MESSAGE) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0
#QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

ADDRESS = CTRL_REG4 & WRITE_CFG
DATA = 0x38
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,MESSAGE) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0
#QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

QSPI_CORE.write(SPICR,0x1C6) #reset Fifo, enable QSPI core in master mode, manual SS, master inhibit


while( QSPI_CORE.read(SPICR) != 0x186): # check to see if there is data
    pass
print('Reset complete...')
#read from WHO_AM_I
ADDRESS = GYRO_WHO_AM_I | READ_CFG
DATA = 0x00
MESSAGE = (ADDRESS << 8) | DATA ### Testing read from SPI Core


QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,MESSAGE) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0

while( (QSPI_CORE.read(0x64) & 0x1) == 1): # check to see if there is data
    pass
#print('')
#Temp1 = (QSPI_CORE.read(SPI_DRR) & 0x00FF)
#print(Temp1)

print((QSPI_CORE.read(SPI_DRR) & 0x00FF))
#print(QSPI_CORE.read(SPI_DRR))

print('DONE')

Reset complete...
104
DONE


In [ ]:
#read from WHO_AM_I
ADDRESS = GYRO_WHO_AM_I | READ_CFG
DATA = 0x00
MESSAGE = (ADDRESS << 8) | DATA ### Testing read from SPI Core

#QSPI_CORE.write(WRITE_OFFSET, MESSAGE) # try to send data

QSPI_CORE.write(0x60, 0x186) # enable QSPI core in master mode, manual SS, master inhibit
QSPI_CORE.write(0x68,MESSAGE) # Data
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0
QSPI_CORE.write(0x60, 0x86) # enable QSPI core in master mode, manual SS
QSPI_CORE.write(0x70, 0xFFFF) # enable SS 0


In [ ]:
print(QSPI_CORE.read(SPI_DRR))

In [16]:
TEST_READ0 = QSPI_CORE.read(0x78) # Recieve FIFO Occupancy (returns n-1 occupancy)
print(TEST_READ0, 'Receive FIFO Occupancy')

#TEST_READ = QSPI_CORE.read(0x6C) #Read from FIFO
#print(TEST_READ, TEST_READ & 0x00FF, np.int16(TEST_READ), 'Read from FIFO')

TEST_READ100 = QSPI_CORE.read(0x64) #this bit is set High when the receive FIFO is empty
print(TEST_READ100 & 0x1, 'High when the receive FIFO is empty')


0 Receive FIFO Occupancy
1 High when the receive FIFO is empty


In [ ]:
# testing read WHO_AM_I
GYRO_WHO_AM_I = 0x0F

# writing read command to PMODNav WHO_AM_I register 0x0F
WRITE_OFFSET = 0x68
ADDRESS = GYRO_WHO_AM_I | READ_CFG
DATA = 0x00
MESSAGE = (ADDRESS << 8) | DATA ### Testing read from SPI Core
QSPI_CORE.write(WRITE_OFFSET, MESSAGE) # try to send data

# reading response from PMODNav from SPI core
READ_OFFSET = 0x6C #SPI_DRR, SPI data reveive

r1 = QSPI_CORE.read(READ_OFFSET)
#r2 = SPI_CORE.read(0x7400)

print('offset: {0:x}\t'.format(READ_OFFSET) + 'data: {0:x}'.format(r1))
#print('offset: {0:x}\t'.format(0x7400) + 'data: {0:x}'.format(r2))

### Testing data status of Nav sensor, i.e. acc and gyro data in FIFO

In [ ]:
# testing status to check acc and gyro data
###### NOT READY FOR USE #########
STATUS_REG1 = 0X17
STATUS_REG2 = 0x27


# writing read command to PMODNav WHO_AM_I register 0x0F
WRITE_OFFSET = 0x68
ADDRESS = GYRO_WHO_AM_I | READ_CFG
DATA = 0x00
MESSAGE = (ADDRESS << 8) | DATA ### Testing read from SPI Core
QSPI_CORE.write(WRITE_OFFSET, MESSAGE) # try to send data

# reading response from PMODNav from SPI core
READ_OFFSET = 0x6C #SPI_DRR, SPI data reveive

r1 = QSPI_CORE.read(READ_OFFSET)
#r2 = SPI_CORE.read(0x7400)

print('offset: {0:x}\t'.format(READ_OFFSET) + 'data: {0:x}'.format(r1))
#print('offset: {0:x}\t'.format(0x7400) + 'data: {0:x}'.format(r2))

## DEMO Running HLS AXI SPI Driver code once

In [ ]:
# test code

# setting ap_start/auto reset on SPI Driver
#SPI_DRIVER.write(0x00,0x81) #runs continuously, see xax_spi_driver_hw.h
if(driverStateDemo < 9):
    driverStateDemo = driverStateDemo + 1
elif(navDataStateDemo > 23):
    navDataStateDemo = 0;
elif(driverStateDemo > 8):
    navDataStateDemo = navDataStateDemo + 1

    
#if(driverStateDemo==5):
    #QSPI_CORE.write(SPICR,0x46) # configure the QSPI along with resetting the FIFO (do right before a read)
print('Calling SPI_DRIVER.write(0x00,0x01)\n')
print('Driver State: ', driverStateDemo)
print('Nav Data State: ', navDataStateDemo)
# setting up Driver core

#SPI_DRIVER.write(0x00,0x01) #runs once
SPI_DRIVER.write(0x00,0x81) #runs continuously, see xax_spi_driver_hw.h

#print('Calling SPI_Driver.read(0x00):')
#READ = SPI_DRIVER.read(0x00)
#print(READ)

print('done')

## Read from SPI_DATA (output of custom HLS AXI SPI Driver core)

In [ ]:
# defining MMIO index offsets
index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14
index_6 = 0x18
index_7 = 0x1C
index_8 = 0x20
index_9 = 0x24
index_10 = 0x28
index_11 = 0x2C
index_12 = 0x30
index_13 = 0x34
index_14 = 0x38
index_15 = 0x3C
index_16 = 0x40
index_17 = 0x44
index_18 = 0x48
index_19 = 0x4C
index_20 = 0x50
index_21 = 0x54
index_22 = 0x58
index_23 = 0x5C
index_24 = 0x60
index_25 = 0x64
index_26 = 0x68
index_27 = 0x6C
index_28 = 0x70
index_29 = 0x74
print('')
print('done')


SPICR = 0x60
DATA_MASK_SHORT = 0x0000FFFF

In [ ]:
#Debug code for *41.bit, *42.bit

# Been running below line of code in Driver State 5
#QSPI_CORE.write(SPICR,0x46) # configure the QSPI along with resetting the FIFO (do right before a read)

TEST_READ0 = QSPI_CORE.read(0x78) # Recieve FIFO Occupancy (returns n-1 occupancy)
print(TEST_READ0, 'Receive FIFO Occupancy')

#TEST_READ = QSPI_CORE.read(0x6C) #Read from FIFO
#print(TEST_READ, TEST_READ & 0x00FF, np.int16(TEST_READ), 'Read from FIFO')

TEST_READ100 = QSPI_CORE.read(0x64) #this bit is set High when the receive FIFO is empty
print(TEST_READ100 & 0x1, 'High when the receive FIFO is empty')


print('\n--- Reading Raw Nav Data --')
# Acceleromoter 
TEST_READ1 = SPI_DATA.read(index_0)
TEST_READ2 = SPI_DATA.read(index_1)
TEST_READ3 = SPI_DATA.read(index_2)

TEST_READ4 = SPI_DATA.read(index_3)
TEST_READ5 = SPI_DATA.read(index_4)
TEST_READ6 = SPI_DATA.read(index_5)

TEST_READ7 = SPI_DATA.read(index_6)
TEST_READ8 = SPI_DATA.read(index_7)
TEST_READ9 = SPI_DATA.read(index_8)
print('accX: ')
print('tempLow: ', TEST_READ1, 'tempHigh: ', TEST_READ2, 'tempCombined: ', TEST_READ3)
print('tempCombined (int16): ', np.int16(TEST_READ3),'\n')
print('accY: ')
print('tempLow: ', TEST_READ4, 'tempHigh: ', TEST_READ5, 'tempCombined: ', TEST_READ6)
print('tempCombined (int16): ', np.int16(TEST_READ6),'\n')
print('accZ: ')
print('tempLow: ', TEST_READ7, 'tempHigh: ', TEST_READ8, 'tempCombined: ', TEST_READ9)
print('tempCombined (int16): ', np.int16(TEST_READ9),'\n')
#print('accX: ', TEST_READ1, 'accY: ', TEST_READ2, 'accZ: ', TEST_READ3)
#Gyroscope
TEST_READ10 = SPI_DATA.read(index_9)
TEST_READ11 = SPI_DATA.read(index_10)
TEST_READ12 = SPI_DATA.read(index_11)

TEST_READ13 = SPI_DATA.read(index_12)
TEST_READ14 = SPI_DATA.read(index_13)
TEST_READ15 = SPI_DATA.read(index_14)

TEST_READ16 = SPI_DATA.read(index_15)
TEST_READ17 = SPI_DATA.read(index_16)
TEST_READ18 = SPI_DATA.read(index_17)
print('gyroX: ')
print('tempLow: ', TEST_READ10, 'tempHigh: ', TEST_READ11, 'tempCombined: ', TEST_READ12)
print('tempCombined (int16): ', np.int16(TEST_READ12),'\n')
print('gyroY: ')
print('tempLow: ', TEST_READ13, 'tempHigh: ', TEST_READ14, 'tempCombined: ', TEST_READ15)
print('tempCombined (int16): ', np.int16(TEST_READ15),'\n')
print('gyroZ: ')
print('tempLow: ', TEST_READ16, 'tempHigh: ', TEST_READ17, 'tempCombined: ', TEST_READ18)
print('tempCombined (int16): ', np.int16(TEST_READ18),'\n')

#print('\n--- Reading Int Raw Nav Data --')
#accX = np.int16(TEST_READ1 & DATA_MASK_SHORT)
#accY = np.int16(TEST_READ2 & DATA_MASK_SHORT)
#accZ = np.int16(TEST_READ3 & DATA_MASK_SHORT)
#gyroX = np.int16(TEST_READ4 & DATA_MASK_SHORT)
#gyroY = np.int16(TEST_READ5 & DATA_MASK_SHORT)
#gyroZ = np.int16(TEST_READ6 & DATA_MASK_SHORT)
#print('accX: ', accX, 'accY: ', accY, 'accZ: ', accZ)
#print('gyroX: ',gyroX,  'gyroY: ', gyroY, 'gyroZ: ', gyroZ)


print('\n--- State Machine Status--')
TEST_READ21 = SPI_DATA.read(index_20)
TEST_READ22 = SPI_DATA.read(index_21)
print('Driver State: ', TEST_READ22)
print('Nav Data State: ', TEST_READ21)

print(' ')
print('done')

In [ ]:
#Debug code for *37.bit, *39.bit

# Been running below line of code in Driver State 5
#QSPI_CORE.write(SPICR,0x46) # configure the QSPI along with resetting the FIFO (do right before a read)

TEST_READ0 = QSPI_CORE.read(0x78) # Recieve FIFO Occupancy (returns n-1 occupancy)
print(TEST_READ0, 'Receive FIFO Occupancy')

#TEST_READ = QSPI_CORE.read(0x6C) #Read from FIFO
#print(TEST_READ, TEST_READ & 0x00FF, np.int16(TEST_READ), 'Read from FIFO')

TEST_READ10 = QSPI_CORE.read(0x64) #this bit is set High when the receive FIFO is empty
print(TEST_READ10 & 0x1, 'High when the receive FIFO is empty')


print('\n--- Reading Raw Nav Data --')
# Acceleromoter 
TEST_READ1 = SPI_DATA.read(index_0)
TEST_READ2 = SPI_DATA.read(index_1)
TEST_READ3 = SPI_DATA.read(index_2)
print('accX: ', TEST_READ1, 'accY: ', TEST_READ2, 'accZ: ', TEST_READ3)
#print('accX: ', TEST_READ1, 'accY: ', TEST_READ2, 'accZ: ', TEST_READ3)
#Gyroscope
TEST_READ4 = SPI_DATA.read(index_3)
TEST_READ5 = SPI_DATA.read(index_4)
TEST_READ6 = SPI_DATA.read(index_5)
print('gyroX: ',TEST_READ4,  'gyroY: ', TEST_READ5, 'gyroZ: ', TEST_READ6)


print('\n--- Reading Int Raw Nav Data --')
accX = np.int16(TEST_READ1 & DATA_MASK_SHORT)
accY = np.int16(TEST_READ2 & DATA_MASK_SHORT)
accZ = np.int16(TEST_READ3 & DATA_MASK_SHORT)
gyroX = np.int16(TEST_READ4 & DATA_MASK_SHORT)
gyroY = np.int16(TEST_READ5 & DATA_MASK_SHORT)
gyroZ = np.int16(TEST_READ6 & DATA_MASK_SHORT)
print('accX: ', accX, 'accY: ', accY, 'accZ: ', accZ)
print('gyroX: ',gyroX,  'gyroY: ', gyroY, 'gyroZ: ', gyroZ)


print('\n--- State Machine Status--')
TEST_READ7 = SPI_DATA.read(index_6)
TEST_READ8 = SPI_DATA.read(index_7)
print('Driver State: ', TEST_READ8)
print('Nav Data State: ', TEST_READ7)

print(' ')
print('done')

In [ ]:
#Debug code for demo

# Been running below line of code in Driver State 5
#QSPI_CORE.write(SPICR,0x46) # configure the QSPI along with resetting the FIFO (do right before a read)

TEST_READ0 = QSPI_CORE.read(0x78) # Recieve FIFO Occupancy (returns n-1 occupancy)
print(TEST_READ0, 'Receive FIFO Occupancy')

#TEST_READ = QSPI_CORE.read(0x6C) #Read from FIFO
#print(TEST_READ, TEST_READ & 0x00FF, np.int16(TEST_READ), 'Read from FIFO')

TEST_READ10 = QSPI_CORE.read(0x64) #this bit is set High when the receive FIFO is empty
print(TEST_READ10 & 0x1, 'High when the receive FIFO is empty')


print('\n--- Reading Raw Nav Data --')
# Acceleromoter 
TEST_READ1 = SPI_DATA.read(index_0)
TEST_READ2 = SPI_DATA.read(index_1)
TEST_READ3 = SPI_DATA.read(index_2)
print('accX: ', TEST_READ1 & 0x00FF, 'accY: ', TEST_READ2 & 0x00FF, 'accZ: ', TEST_READ3 & 0x00FF)
#print('accX: ', TEST_READ1, 'accY: ', TEST_READ2, 'accZ: ', TEST_READ3)
#Gyroscope
TEST_READ4 = SPI_DATA.read(index_3)
TEST_READ5 = SPI_DATA.read(index_4)
TEST_READ6 = SPI_DATA.read(index_5)
print('gyroX: ',TEST_READ4 & 0x00FF,  'gyroY: ', TEST_READ5 & 0x00FF, 'gyroZ: ', TEST_READ6 & 0x00FF)

print('\n--- State Machine Status--')
TEST_READ7 = SPI_DATA.read(index_6)
TEST_READ8 = SPI_DATA.read(index_7)
print('Driver State: ', TEST_READ8)
print('Nav Data State: ', TEST_READ7)

print(' ')
print('done')

In [ ]:
#Parameter dps range : +/- 245dps
NAV_GYRO_PAR_G_245DPS = 0
#Parameter dps range : +/- 500dps
NAV_GYRO_PAR_G_500DPS = 1
#Parameter dps range : +/- 2kdps
NAV_GYRO_PAR_G_2kDPS = 3

#Parameter G range : +/- 2g
NAV_ACL_PAR_XL_2G  = 0
#Parameter G range : +/- 4g
NAV_ACL_PAR_XL_4G = 2 
#Parameter G range : +/- 8g
NAV_ACL_PAR_XL_8G = 3
#Parameter G range : +/- 16g
NAV_ACL_PAR_XL_16G  = 1

# creating conversion math for bit precision
def Nav_GetGRangeLSB(bRangeG):
    gRangeLSB = 0
    if(bRangeG==0): 
    #case NAV_GYRO_PAR_G_245DPS:
        gRangeLSB = 0.00875
    elif(bRangeG==1):
    #case NAV_GYRO_PAR_G_500DPS:
        gRangeLSB = 0.0175
    elif(bRangeG==2):
    #case NAV_GYRO_PAR_G_2kDPS:
        gRangeLSB = 0.07
    else:
        gRangeLSB = 0.00875
    print('Selecting gRangeLSB: %f\n' % gRangeLSB)
    return gRangeLSB

def Nav_GetXLRangeLSB(bRangeXL):
    xlRangeLSB = 0
    if(bRangeXL==0):
#    case NAV_ACL_PAR_XL_2G
        xlRangeLSB = 0.000061
    elif(bRangeXL==1):
#  case NAV_ACL_PAR_XL_4G
        xlRangeLSB = 0.000122
    elif(bRangeXL==1):
#   case NAV_ACL_PAR_XL_8G
        xlRangeLSB = 0.000244
    elif(bRangeXL==2):
#   case NAV_ACL_PAR_XL_16G
        xlRangeLSB = 0.000732
    else:
        xlRangeLSB = 0.000061
    print('Selecting xlRangeLSB: %f\n' % xlRangeLSB)
    return xlRangeLSB

In [ ]:
gRange = Nav_GetGRangeLSB(0)
xlRange = Nav_GetXLRangeLSB(0)

T_SAMPLE = 50
FS = 2
N_SAMPLE = T_SAMPLE*FS

ACC_RAW = list()
GYRO_RAW = list()
ACC_DATA = list()
GYRO_DATA = list()

for x in range(0, N_SAMPLE):
    # Raw Data
    # Acceleromoter 
    TEST_READ1 = SPI_DATA.read(index_2)
    TEST_READ2 = SPI_DATA.read(index_5)
    TEST_READ3 = SPI_DATA.read(index_8)
    #Gyroscope
    TEST_READ4 = SPI_DATA.read(index_11)
    TEST_READ5 = SPI_DATA.read(index_14)
    TEST_READ6 = SPI_DATA.read(index_17)
    
    #ACC_RAW.append(np.int16(TEST_READ1))
    #ACC_RAW.append(np.int16(TEST_READ2))
    #ACC_RAW.append(np.int16(TEST_READ3))
    accXRaw = np.int16(TEST_READ1 & DATA_MASK_SHORT)
    accYRaw = np.int16(TEST_READ2 & DATA_MASK_SHORT)
    accZRaw = np.int16(TEST_READ3 & DATA_MASK_SHORT)
    
    ACC_RAW.append(accXRaw)
    ACC_RAW.append(accYRaw)
    ACC_RAW.append(accZRaw)
    
    #GYRO_RAW.append(np.int16(TEST_READ4))
    #GYRO_RAW.append(np.int16(TEST_READ5))
    #GYRO_RAW.append(np.int16(TEST_READ6))
    
    gyroXRaw = np.int16(TEST_READ4 & DATA_MASK_SHORT)
    gyroYRaw = np.int16(TEST_READ5 & DATA_MASK_SHORT)
    gyroZRaw = np.int16(TEST_READ6 & DATA_MASK_SHORT)
    
    GYRO_RAW.append(gyroXRaw)
    GYRO_RAW.append(gyroYRaw)
    GYRO_RAW.append(gyroZRaw)
    
    accX = accXRaw * xlRange
    accY = accYRaw * xlRange
    accZ = accZRaw * xlRange
    
    gyroX = gyroXRaw * gRange
    gyroY = gyroYRaw * gRange
    gyroZ = gyroZRaw * gRange
    
    ACC_DATA.append(accX)
    ACC_DATA.append(accY)
    ACC_DATA.append(accZ)
    
    GYRO_DATA.append(gyroX)
    GYRO_DATA.append(gyroY)
    GYRO_DATA.append(gyroZ)
    
    print('Raw accX: ', accXRaw, 'accY: ', accYRaw, 'accZ: ', accZRaw, '\r\n')
    print('Raw gyroX: ', gyroXRaw,  'gyroY: ', gyroYRaw, 'gyroZ: ', gyroZRaw, '\r\n')
    
    print('Data accX: ', ACC_DATA[(x*3) + 0], 'accY: ', ACC_DATA[(x*3) + 1], 'accZ: ', ACC_DATA[(x*3) + 2], '\r\n')
    print('Data gyroX: ',GYRO_DATA[(x*3) + 0], 'gyroY: ',GYRO_DATA[(x*3) + 1], 'gyroZ: ',GYRO_DATA[(x*3) + 2], '\r\n')
    time.sleep(1/FS)

In [ ]:
ACC_RAW

## Plot Nav Data

In [ ]:
import matplotlib.pyplot as plt

RAD_TO_DEG = 180./np.pi

#def packRawGyroData():
#    return [Nav_Gyro_GetData(0x01), Nav_Gyro_GetData(0x02), 
#            Nav_Gyro_GetData(0x03)]

#def packRawAccData():
#    return [Nav_Acc_GetData(0x01), Nav_Acc_GetData(0x02), 
#            Nav_Acc_GetData(0x03)]

#N = 200 # Number of iterations
#GyroRawNavData = np.array([packRawGyroData() for _ in range(N)])
#print("Gyroscope Raw Data: \n", GyroRawNavData)
#GyroDPSNavData = Gyro_gRangeLSB * GyroRawNavData
#print("Gyroscope DPS Data: \n", GyroDPSNavData)

# Might need to flip values i.e. multiply by (-1), decide based on testing and reference wanted
#AccRawNavData = np.array([packRawAccData() for _ in range(N)])
#print("Accelerometer Raw Data: \n", AccRawNavData)
#AccGNavData = Accel_xlRangeLSB * AccRawNavData
#print("Accelerometer G Data: \n", AccGNavData)

GYRO_RAW_X = np.zeros(N_SAMPLE)
GYRO_RAW_Y = np.zeros(N_SAMPLE)
for x in range(0, N_SAMPLE):
    GYRO_RAW_X[x] = GYRO_DATA[(x*3) + 0]
    GYRO_RAW_Y[x] = GYRO_DATA[(x*3) + 1]
plt.figure()
plt.plot(GYRO_RAW_X , label = 'Roll Rate')
plt.plot(GYRO_RAW_Y, label = 'Pitch Rate')
#plt.plot(GyroDPSNavData[:, 2], label = 'Yaw Rate')
plt.ylabel('Dps')
plt.xlabel('Counts')
plt.title('Gyro DPS Data')
plt.legend(loc=2)


AccRawNavData_X = np.zeros(N_SAMPLE)
AccRawNavData_Y = np.zeros(N_SAMPLE)
AccRawNavData_Z = np.zeros(N_SAMPLE)
# Convert Accelerometer values to degrees
for x in range(0,N_SAMPLE):
    ax = ACC_RAW[(x*3) + 0]
    ay = ACC_RAW[(x*3) + 1]
    az = ACC_RAW[(x*3) + 2]
    AccRawNavData_X[x] = ax
    AccRawNavData_Y[x] = ay
    AccRawNavData_Z[x] = az
rollAccAngle = np.arctan2(AccRawNavData_Y, AccRawNavData_Z + np.pi) * RAD_TO_DEG
pitchAccAngle = np.arctan2(AccRawNavData_X, AccRawNavData_Z + np.pi) * RAD_TO_DEG

plt.figure()
plt.plot(rollAccAngle, label = 'Roll Accelerometer Angle')
plt.plot(pitchAccAngle, label = 'Pitch Accelerometer Angle')
plt.ylabel('Degrees')
plt.xlabel('Counts')
plt.title('Roll and Pitch Angles from Accelerometer')
plt.legend(loc=2)

plt.figure()
plt.plot(AccRawNavData_X, label = 'AccRawNavData_X')
plt.plot(AccRawNavData_Y, label = 'AccRawNavData_Y')
plt.ylabel('G')
plt.xlabel('Counts')
plt.title('Accelerometer')
plt.legend(loc=2)

In [ ]:
GYRO_RAW_X

In [ ]:
pitchAccAngle

## Debug plotting

In [ ]:
# Debug code only
#QSPI_CORE.write(SPICR,0x46) # configure the QSPI along with resetting the FIFO (do right before a read)

TEST_READ0 = QSPI_CORE.read(0x78) # Recieve FIFO Occupancy (returns n-1 occupancy)
print(TEST_READ0, 'Receive FIFO Occupancy')

#TEST_READ = QSPI_CORE.read(0x6C) #Read from FIFO
#print(TEST_READ, TEST_READ & 0x00FF, np.int16(TEST_READ), 'Read from FIFO')

#f = 65286
#print(f, np.int16(f),'FIFO data recorded in int16t')
#SPI_DATA.write(index_1,69) # 32 bit version, see HLS

TEST_READ10 = QSPI_CORE.read(0x64) #this bit is set High when the receive FIFO is empty
print(TEST_READ10 & 0x1, 'High when the receive FIFO is empty')


print('\n--- Reading Data --')
TEST_READ1 = SPI_DATA.read(index_0)# 32 bit version, see HLS
print(TEST_READ1, TEST_READ1 & 0x00FF, np.int16(TEST_READ1), 'SPI_DATA.read(index_0)')

TEST_READ2 = SPI_DATA.read(index_1)# 32 bit version, see HLS
print(TEST_READ2, TEST_READ2 & 0x00FF, np.int16(TEST_READ2), 'accY')

TEST_READ3 = SPI_DATA.read(index_2)# 32 bit version, see HLS
print(TEST_READ3, TEST_READ3 & 0x00FF, np.int16(TEST_READ3), 'accZ')

TEST_READ4 = SPI_DATA.read(index_3)# 32 bit version, see HLS
print(TEST_READ4, TEST_READ4 & 0x00FF, np.int16(TEST_READ4), 'gyroX')

TEST_READ5 = SPI_DATA.read(index_4)# 32 bit version, see HLS
print(TEST_READ5, TEST_READ5 & 0x00FF, np.int16(TEST_READ5), 'gyroY')

TEST_READ6 = SPI_DATA.read(index_5)# 32 bit version, see HLS
print(TEST_READ6, TEST_READ6 & 0x00FF, np.int16(TEST_READ6), 'gyroZ')


print('\n\n -- DEBUG ONLY, not valid data --')
TEST_READ7 = SPI_DATA.read(index_6)# 32 bit version, see HLS
print(TEST_READ7,'Nav Data State','SPI_DATA.read(index_6)')

TEST_READ8 = SPI_DATA.read(index_7)# 32 bit version, see HLS
print(TEST_READ8,'State')

TEST_READ9 = SPI_DATA.read(index_8)# 32 bit version, see HLS
print(TEST_READ9)

print(' ')
print('done')


Seeing a return of (x)    (x & 0x00FF), np.int16(x)
"                 "65535, 255, -1 Read from FIFO, from xspi_write(CTRL_REG5_XL, 0x38)
"                 "65535, 255, -1 Read from FIFO, from xspi_write(CTRL_REG6_XL, 0xD8)
"                 "65535, 255, -1 Read from FIFO, from xspi_write(CTRL_REG1_G, 0xDB)
"                 "65535, 255, -1 Read from FIFO, from xspi_write(CTRL_REG4, 0x38)

MMIO Debug: Reading 4 bytes from offset 78
0 Receive FIFO Occupancy
MMIO Debug: Reading 4 bytes from offset 64
1 High when the receive FIFO is empty

--- Reading Data --
MMIO Debug: Reading 4 bytes from offset 0
65288 8 -248 SPI_DATA.read(index_0)
MMIO Debug: Reading 4 bytes from offset 4
65282 2 -254 accY
MMIO Debug: Reading 4 bytes from offset 8
65434 154 -102 accZ
MMIO Debug: Reading 4 bytes from offset c
65321 41 -215 gyroX
MMIO Debug: Reading 4 bytes from offset 10
65282 2 -254 gyroY
MMIO Debug: Reading 4 bytes from offset 14
65312 32 -224 gyroZ


 -- DEBUG ONLY, not valid data --
MMIO Debug: Reading 4 bytes from offset 18
11 Nav Data State SPI_DATA.read(index_6)
MMIO Debug: Reading 4 bytes from offset 1c
8 State
MMIO Debug: Reading 4 bytes from offset 20
17476
 
done

### Testing SPI vein integrated with AXI Quad SPI, AXI_SPI_Driver (custom), and positionCtrl (custom)

In [ ]:

# HLS test write
#UART_Data.write(0x00,0x45)
SPI_DATA.write(0x0000,0x0045)
POS_DATA.write(0x0000,0x0045)
POS_TEST.write(0x0000,0x0045)

# HLS test read
TEST_READ1 = SPI_DATA.read(0x0000)
TEST_READ2 = POS_DATA.read(0x0000)
TEST_READ3 = POS_TEST.read(0x0000)

print('Test Data: ')
print(TEST_READ1)
print('')
print('Test Data: ')
print(TEST_READ2)
print('')
print('Test Data: ')
print(TEST_READ3)
print('')

print('')
print('done')